Round 255 - prediction results for ensemble model of catboost and lgbmboost. Nothing fancy as of yet. We build on round 254 lgbm by using ensemble model with lgbm + cat. Need to setup proper pipeline for downloading data and 

In [14]:
import sys
sys.path.append("..") 


import os
import argparse
import pickle
import json
from tqdm import tqdm

import numpy as np
import pandas as pd
import numerapi
from getpass import getpass
import seaborn as sns
#from dotenv import load_dotenv, find_dotenv
import matplotlib.pyplot as plt
from pathlib import Path

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mse
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

import utilities
import evaluation
from autoencoder import AutoEncoder

In [6]:
os.environ["NUMERAI_PUBLIC_KEY"] = getpass("Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ")
os.environ["NUMERAI_SECRET_KEY"] = getpass("Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ")
os.environ["NUMERAI_MODEL_ID"] = getpass("Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ")

public_key = os.environ.get("NUMERAI_PUBLIC_KEY")
secret_key = os.environ.get("NUMERAI_SECRET_KEY")
model_id = os.environ.get("NUMERAI_MODEL_ID")
napi = numerapi.NumerAPI(verbosity="info", public_id=public_key, secret_key=secret_key)

Please enter your Numerai Public Key. You can find your key here: https://numer.ai/submit -> ········
Please enter your Numerai Secret Key. You can find your key here: https://numer.ai/submit -> ········
Please enter your Numerai Model ID. You can find your key here: https://numer.ai/submit -> ········


In [7]:
napi.download_current_dataset()

./numerai_dataset_254.zip: 100%|█████████▉| 394M/394M [15:45<00:00, 580kB/s]     2021-03-08 13:30:34,435 INFO numerapi.base_api: unzipping file...


'./numerai_dataset_254.zip'

In [15]:
tourn_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_tournament_data.csv')
train_file = Path(f'./numerai_dataset_{napi.get_current_round()}/numerai_training_data.csv')
processed_train_file = Path('./training_processed.csv')

if processed_train_file.exists():
    print("Loading the processed training data from file\n")
    training_data = pd.read_csv(processed_train_file)
else:
    tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
    val_df = pd.concat([chunk[chunk['data_type'] == 'validation'] for chunk in tqdm(tourn_iter_csv)])
    tourn_iter_csv.close()
    training_data = pd.read_csv(train_file)
    training_data = pd.concat([training_data, val_df])
    training_data.reset_index(drop=True, inplace=True)
    print("Training Dataset Generated! Saving to file ...")
    training_data.to_csv(processed_train_file, index=False)


feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]
target_cols = ['target']

train_idx = training_data.index[training_data.data_type=='train'].tolist()
val_idx = training_data.index[training_data.data_type=='validation'].tolist()

2it [01:17, 38.55s/it]


Training Dataset Generated! Saving to file ...


In [17]:
from tensorflow.keras.models import load_model

model=load_model('autoencoder')

In [19]:
for layer in model.layers:
  if 'dense' in layer.get_config()['name']:
    if layer.get_config()['units']==50:
      output=layer.output

In [20]:
from tensorflow.keras.models import Model
encoder=Model(inputs=model.input, outputs=output)

In [23]:
#cat_model=pickle.load(open('cat_model_round_254.pkl', 'rb'))
lgb_model=pickle.load(open('lgbm_model_round_254.pkl', 'rb'))

In [24]:
ids = []
preds = []

chunksize = 5000

tourn_iter_csv = pd.read_csv(tourn_file, iterator=True, chunksize=1e6)
for chunk in tourn_iter_csv:
    live_data = chunk[feature_cols].to_numpy()
    x_live_ae = encoder.predict(live_data)
    out = lgb_model.predict(x_live_ae)
    ids.extend(chunk["id"])
    preds.extend(out)
tourn_iter_csv.close()

In [26]:
predictions_df = pd.DataFrame({
    'id':ids,
    'prediction':preds
})
predictions_df.head()

,id,prediction
0,n0003aa52cab36c2,0.483834
1,n000920ed083903f,0.492721
2,n0038e640522c4a6,0.519278
3,n004ac94a87dc54b,0.497608
4,n0052fe97ea0c05f,0.494105


In [27]:
predictions_df.to_csv("lgb_predictions.csv", index=False)